In [1]:
>>> from urllib.request import urlopen

In [3]:
>>> url = "http://olympus.realpython.org/profiles/aphrodite"

In [5]:
>>> page = urlopen(url)

In [11]:
>>> html_bytes = page.read()
>>> html = html_bytes.decode("utf-8")

In [21]:
!pip install earthengine-api
!pip install jupyter


   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   -------------- ------------------------- 4.7/12.7 MB 31.6 MB/s eta 0:00:01
   ----------------------------- ---------- 9.4/12.7 MB 22.5 MB/s eta 0:00:01
   ---------------------------------- ----- 11.0/12.7 MB 20.2 MB/s eta 0:00:01
   -------------------------------------- - 12.3/12.7 MB 15.7 MB/s eta 0:00:01
   ---------------------------------------- 12.7/12.7 MB 13.8 MB/s eta 0:00:00


In [23]:
import ee

# Authenticate your Google Earth Engine account
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize()

Enter verification code:  4/1AanRRrtxGrvYeeEdcaZab6A3mlrNd8n3hA1dit8K2gelCXnqGEGtTG9GOpk



Successfully saved authorization token.


In [24]:
# Load a dataset: Global Surface Water
dataset = ee.Image('JRC/GSW1_3/GlobalSurfaceWater')
print(dataset.getInfo())


C:\Users\4587sajid\AppData\Local\anaconda3\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_3/GlobalSurfaceWater! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_3_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)


{'type': 'Image', 'bands': [{'id': 'occurrence', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [1335834, 545466], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -180.00001488697754, 0, -0.00026949458523585647, 84.00011474509029]}, {'id': 'change_abs', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [1335834, 545466], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -180.00001488697754, 0, -0.00026949458523585647, 84.00011474509029]}, {'id': 'change_norm', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [1335834, 545466], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -180.00001488697754, 0, -0.00026949458523585647, 84.00011474509029]}, {'id': 'seasonality', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [1335834, 545466], 'crs': 'EPSG:4326', 'crs_

In [27]:
!pip install folium


In [29]:
import folium

# Define a location to center the map
map_center = [0, 0]  # Latitude, Longitude

# Create a folium map
m = folium.Map(location=map_center, zoom_start=2)

# Add Earth Engine layer (e.g., Global Surface Water)
def add_ee_layer(map_object, ee_image, vis_params, name):
    map_id_dict = ee.Image(ee_image).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Visualization parameters
vis_params = {
    'bands': ['occurrence'],
    'min': 0,
    'max': 100,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Add the layer to the map
add_ee_layer(m, dataset, vis_params, 'Global Surface Water')

# Add layer control and display the map
m.add_child(folium.LayerControl())
m


In [35]:
# Define a region of interest as a list of coordinates (in GeoJSON format)
region = [
    [72.0, 18.0],  # Bottom-left
    [73.0, 18.0],  # Bottom-right
    [73.0, 19.0],  # Top-right
    [72.0, 19.0],  # Top-left
    [72.0, 18.0]   # Close the polygon
]

# Export to Google Drive
export_task = ee.batch.Export.image.toDrive(
    image=dataset,
    description='GlobalSurfaceWater',
    folder='GEE_Exports',
    scale=30,
    region=region
)

# Start the task
export_task.start()


In [37]:
!pip install googlemaps


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40747 sha256=a049561a752ad00a468b485ec48dff61cb3dfa0692356a33d8d833c5a3267522
  Stored in directory: c:\users\4587sajid\appdata\local\pip\cache\wheels\4c\6a\a7\bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


In [39]:
import googlemaps
import csv

# Initialize the Google Maps client with your API key
API_KEY = "4/1AanRRrtxGrvYeeEdcaZab6A3mlrNd8n3hA1dit8K2gelCXnqGEGtTG9GOpk"
gmaps = googlemaps.Client(key=API_KEY)

# Define the search parameters
location = (24.8607, 67.0011)  # Karachi coordinates (latitude, longitude)
radius = 10000  # Search within 10km radius
type_place = "school"  # Type of place to search

# Perform the search
results = gmaps.places_nearby(location=location, radius=radius, type=type_place)

# Extract data
schools = []
for place in results.get("results", []):
    schools.append({
        "name": place.get("name"),
        "address": place.get("vicinity"),
        "latitude": place["geometry"]["location"]["lat"],
        "longitude": place["geometry"]["location"]["lng"],
    })

# Save data to CSV
with open("karachi_schools.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["name", "address", "latitude", "longitude"])
    writer.writeheader()
    writer.writerows(schools)

print("Data saved to karachi_schools.csv")


ValueError: Invalid API key provided.

In [41]:
curl -X GET "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=24.8607,67.0011&radius=1000&type=school&key=YOUR_API_KEY"


SyntaxError: invalid syntax (3842632429.py, line 1)

In [43]:
!pip install beautifulsoup4

In [45]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = "https://www.google.com/maps/search/schools+in+karachi/@24.8533469,66.9707992,12z/data=!3m1!4b1?entry=ttu&g_ep=EgoyMDI0MTIxMS4wIKXMDSoASAFQAw%3D%3D"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [47]:
>>> print(soup.get_text())


      Google Maps                                           When you have eliminated the JavaScript, whatever remains must be an empty page.   Enable JavaScript to see Google Maps.      


In [49]:
>>> soup.find_all("img")

[]

In [ ]:
==========================================================================================================================

In [51]:
import requests
from bs4 import BeautifulSoup

url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"

# Send a GET request
response = requests.get(url)

In [53]:
import requests
from bs4 import BeautifulSoup

url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"

# Send a GET request
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Example: Find all table rows in a specific table
    table = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_gvSchools'})  # Adjust table ID
    if table:
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            data = [col.text.strip() for col in columns]
            print(data)
    else:
        print("Table not found!")
else:
    print(f"Failed to fetch the webpage: {response.status_code}")

Table not found!


In [55]:
import requests
from bs4 import BeautifulSoup

url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"

# Send a GET request
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Example: Find all table rows in a specific table
    table = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_gvSchools'})  # Adjust table ID
    if table:
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            data = [col.text.strip() for col in columns]
            print(data)
    else:
        print("Table not found!")
else:
    print(f"Failed to fetch the webpage: {response.status_code}")

Table not found!


In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with your WebDriver
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

try:
    # Wait for the page and table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder1_gvSchools"))
    )

    # Scroll to the table if necessary
    table = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_gvSchools")
    ActionChains(driver).move_to_element(table).perform()

    # Extract rows
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        data = [col.text.strip() for col in columns]
        print(data)
    
    # Handle pagination if required
    next_button = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_gvSchools_ctl23_lnkNext")  # Adjust ID
    while next_button:
        next_button.click()
        time.sleep(3)  # Wait for next page to load
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            data = [col.text.strip() for col in columns]
            print(data)
        next_button = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_gvSchools_ctl23_lnkNext")
finally:
    driver.quit()

ModuleNotFoundError: No module named 'selenium'

In [59]:
!pip install selenium


   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 2.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.7 MB 2.6 MB/s eta 0:00:04
   ------- -------------------------------- 1.8/9.7 MB 2.7 MB/s eta 0:00:03
   --------- ------------------------------ 2.4/9.7 MB 2.6 MB/s eta 0:00:03
   ----------- ---------------------------- 2.9/9.7 MB 2.7 MB/s eta 0:00:03
   -------------- ------------------------- 3.4/9.7 MB 2.7 MB/s eta 0:00:03
   ---------------- ----------------------- 3.9/9.7 MB 2.7 MB/s eta 0:00:03
   ------------------ --------------------- 4.5/9.7 MB 2.7 MB/s eta 0:00:02
   -------------------- ------------------- 5.0/9.7 MB 2.7 MB/s eta 0:00:02
   ----------------------- ---------------- 5.8/9.7 MB 2.7 MB/s eta 0:00:02
   ------------------------- -------------- 6.3/9.7 MB 2.7 MB/s eta 0:00:02
   -----------------------

In [65]:
from selenium import webdriver

# Initialize WebDriver for Chrome (Replace with your browser if needed)
driver = webdriver.Chrome()  # Ensure ChromeDriver is in PATH or specify its path
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")
print(driver.title)
driver.quit()


Schools & Employees Information - SE&LD


In [63]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with your WebDriver
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

try:
    # Wait for the page and table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder1_gvSchools"))
    )

    # Scroll to the table if necessary
    table = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_gvSchools")
    ActionChains(driver).move_to_element(table).perform()

    # Extract rows
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        data = [col.text.strip() for col in columns]
        print(data)
    
    # Handle pagination if required
    next_button = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_gvSchools_ctl23_lnkNext")  # Adjust ID
    while next_button:
        next_button.click()
        time.sleep(3)  # Wait for next page to load
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            data = [col.text.strip() for col in columns]
            print(data)
        next_button = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_gvSchools_ctl23_lnkNext")
finally:
    driver.quit()


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF787C6FB05+28789]
	(No symbol) [0x00007FF787BD86E0]
	(No symbol) [0x00007FF787A7592A]
	(No symbol) [0x00007FF787AC930E]
	(No symbol) [0x00007FF787AC95FC]
	(No symbol) [0x00007FF787B128A7]
	(No symbol) [0x00007FF787AEF47F]
	(No symbol) [0x00007FF787B0F654]
	(No symbol) [0x00007FF787AEF1E3]
	(No symbol) [0x00007FF787ABA938]
	(No symbol) [0x00007FF787ABBAA1]
	GetHandleVerifier [0x00007FF787FA933D+3410093]
	GetHandleVerifier [0x00007FF787FBE7DD+3497293]
	GetHandleVerifier [0x00007FF787FB2A73+3448803]
	GetHandleVerifier [0x00007FF787D37BBB+848171]
	(No symbol) [0x00007FF787BE3C3F]
	(No symbol) [0x00007FF787BDF6E4]
	(No symbol) [0x00007FF787BDF87D]
	(No symbol) [0x00007FF787BCED49]
	BaseThreadInitThunk [0x00007FFFB5BB257D+29]
	RtlUserThreadStart [0x00007FFFB74EAF28+40]


In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with webdriver for your browser if not Chrome
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

# Function to extract table data
def scrape_table():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    for row in rows:
        columns = row.find_elements(By.TAG_NAME, "td")
        data.append([col.text.strip() for col in columns])
    return data

try:
    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    # Initialize list to store all table data
    all_data = []
    
    # Scrape the first page
    all_data.extend(scrape_table())
    
    # Handle pagination (if available)
    while True:
        try:
            # Locate and click the "Next" button
            next_button = driver.find_element(By.LINK_TEXT, "Next")
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
            all_data.extend(scrape_table())
        except:
            print("No more pages or pagination not found.")
            break

    # Save data to CSV
    with open("school_data.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerows(all_data)
    print("Data scraped and saved to school_data.csv.")
finally:
    driver.quit()

No more pages or pagination not found.
Data scraped and saved to school_data.csv.


In [69]:
import os
print("File will be saved in:", os.getcwd())


File will be saved in: C:\Users\4587sajid


In [73]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Or any other WebDriver you're using
driver.get("https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=408140046&mne=0")

try:
    # Wait for the map to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "map"))
    )

    # Execute JavaScript to extract map information
    map_center = driver.execute_script("""
        var map = L.map('map');  // Access the Leaflet map instance
        return map.getCenter();  // Get the center coordinates
    """)
    print("Map Center Coordinates:", map_center)

    # Extract marker positions
    markers = driver.execute_script("""
        var markers = [];
        map.eachLayer(function (layer) {
            if (layer instanceof L.Marker) {
                markers.push(layer.getLatLng());
            }
        });
        return markers;  // Return all marker coordinates
    """)
    print("Marker Coordinates:", markers)

finally:
    driver.quit()

JavascriptException: Message: javascript error: Map container is already initialized.
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF77D39FB05+28789]
	(No symbol) [0x00007FF77D3086E0]
	(No symbol) [0x00007FF77D1A592A]
	(No symbol) [0x00007FF77D1AC48F]
	(No symbol) [0x00007FF77D1AF4BF]
	(No symbol) [0x00007FF77D240910]
	(No symbol) [0x00007FF77D21F43A]
	(No symbol) [0x00007FF77D23F654]
	(No symbol) [0x00007FF77D21F1E3]
	(No symbol) [0x00007FF77D1EA938]
	(No symbol) [0x00007FF77D1EBAA1]
	GetHandleVerifier [0x00007FF77D6D933D+3410093]
	GetHandleVerifier [0x00007FF77D6EE7DD+3497293]
	GetHandleVerifier [0x00007FF77D6E2A73+3448803]
	GetHandleVerifier [0x00007FF77D467BBB+848171]
	(No symbol) [0x00007FF77D313C3F]
	(No symbol) [0x00007FF77D30F6E4]
	(No symbol) [0x00007FF77D30F87D]
	(No symbol) [0x00007FF77D2FED49]
	BaseThreadInitThunk [0x00007FFFB5BB257D+29]
	RtlUserThreadStart [0x00007FFFB74EAF28+40]


In [11]:
import requests  # Correct module name
from bs4 import BeautifulSoup

url = "https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=408140046&mne=0"

req = requests.get(url)
print(req)


<Response [200]>


In [19]:
import requests  # Correct module name
from bs4 import BeautifulSoup

url = "https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=408140046&mne=0"

req = requests.get(url)
print(req)

soup = BeautifulSoup(req.content,"html")
print(soup.prettify())

<Response [200]>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Schools &amp; Employees Information - SE&amp;LD
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Online Checker, Online Biometric Checker, Online Biometric, Online Insitution Checker, Online Institution, School and Employees Information, SE&amp;LD, School Education, School Education &amp; Literacy Department, Government of Sindh, GoS, Literacy Department, Sindh" name="keywords"/>
  <link href="../js/bootstrap.min.css" rel="stylesheet"/>
  <script src="http://ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js">
  </script>
  <script src="../js/jquery_3.4.1.min.js">
  </script>
  <script src="../js/3.4.0_bootstrap.min.js">
  </script>
  <link href="../assets/global/plugins/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <style type="text/css">
   .table-header-center {
            

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Set up WebDriver
driver = webdriver.Chrome()  # Ensure ChromeDriver is installed and in PATH
url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"
driver.get(url)

# Wait for the table to load
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_gvInstitutions")))

# Extract table rows
rows = driver.find_elements(By.CSS_SELECTOR, "#ContentPlaceHolder1_gvInstitutions tr")

# Prepare data storage
data = [["Institution ID", "Institution Name", "District", "GPS Coordinates"]]

# Loop through each row (skip the header row)
for i in range(1, len(rows)):
    try:
        # Get Institution ID and other details
        cols = rows[i].find_elements(By.TAG_NAME, "td")
        institution_id = cols[0].text
        institution_name = cols[1].text
        district = cols[2].text

        # Click the Institution ID link to open the details page
        cols[0].find_element(By.TAG_NAME, "a").click()

        # Wait for the details page to load
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "map")))

        # Extract GPS location using JavaScript
        gps_location = driver.execute_script("return gps_location;")
        
        # Append data to the list
        data.append([institution_id, institution_name, district, gps_location])

        # Go back to the main page
        driver.back()

        # Wait for the table to reload
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_gvInstitutions")))

        # Re-fetch rows
        rows = driver.find_elements(By.CSS_SELECTOR, "#ContentPlaceHolder1_gvInstitutions tr")

    except Exception as e:
        print(f"Error processing row {i}: {e}")
        continue

# Save data to CSV
with open("institutions_data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("Data scraping completed. Saved to institutions_data.csv")

# Close the driver
driver.quit()


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF72E8CFB05+28789]
	(No symbol) [0x00007FF72E8386E0]
	(No symbol) [0x00007FF72E6D592A]
	(No symbol) [0x00007FF72E72930E]
	(No symbol) [0x00007FF72E7295FC]
	(No symbol) [0x00007FF72E7728A7]
	(No symbol) [0x00007FF72E74F47F]
	(No symbol) [0x00007FF72E76F654]
	(No symbol) [0x00007FF72E74F1E3]
	(No symbol) [0x00007FF72E71A938]
	(No symbol) [0x00007FF72E71BAA1]
	GetHandleVerifier [0x00007FF72EC0933D+3410093]
	GetHandleVerifier [0x00007FF72EC1E7DD+3497293]
	GetHandleVerifier [0x00007FF72EC12A73+3448803]
	GetHandleVerifier [0x00007FF72E997BBB+848171]
	(No symbol) [0x00007FF72E843C3F]
	(No symbol) [0x00007FF72E83F6E4]
	(No symbol) [0x00007FF72E83F87D]
	(No symbol) [0x00007FF72E82ED49]
	BaseThreadInitThunk [0x00007FFFB5BB257D+29]
	RtlUserThreadStart [0x00007FFFB74EAF28+40]


In [73]:
import requests  # Correct module name
from bs4 import BeautifulSoup

url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"

r = requests.get(url)
print(r)

soup = BeautifulSoup(r.text,"lxml")
print(soup)
np = soup.find("a",class_="btn btn-primary").get("href")
print(np)

#cnp = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"+np

<Response [200]>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
<head><title>
	Schools &amp; Employees Information - SE&amp;LD
</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="Online Checker, Online Biometric Checker, Online Biometric, Online Insitution Checker, Online Institution, School and Employees Information, SE&amp;LD, School Education, School Education &amp; Literacy Department, Government of Sindh, GoS, Literacy Department, Sindh" name="keywords"/><link href="../js/bootstrap.min.css" rel="stylesheet"/>
<script src="http://ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>
<script src="../js/jquery_3.4.1.min.js"></script>
<script src="../js/3.4.0_bootstrap.min.js"></script>
<link href="../assets/global/plugins/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<style type="text/css">
        .table-header-center {
            text-align: center;
        }
    

In [91]:
import requests
from bs4 import BeautifulSoup

# Target URL
url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"

# Send GET request
r = requests.get(url)

# Check for successful request
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "lxml")
    
    # Find the desired anchor tag with class "btn btn-primary"
    link_tag = soup.find("a", class_="btn btn-primary")
    if link_tag:
        # Extract the 'href' attribute (link)
        np = link_tag.get("href")
        print("Extracted Link:", np)
    else:
        print("No link found with the specified class.")


Extracted Link: CheckBiometrics.aspx


In [99]:
import requests
from bs4 import BeautifulSoup

# Target URL
url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"

# Send GET request
r = requests.get(url)

# Check for successful request
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "lxml")
    
    # Find all anchor tags in the table rows (if needed)
    links = soup.find_all("a", href=True)
    
    for link in links:
        # Combine base URL with the relative href
        full_link = requests.compat.urljoin(url, link['href'])
        print("Extracted Link:", full_link)
else:
    print(f"Failed to fetch the page. Status code: {r.status_code}")

Extracted Link: https://checker.sindheducation.gov.pk/ListInstitution.aspx
Extracted Link: https://checker.sindheducation.gov.pk/CheckBiometrics.aspx
Extracted Link: https://checker.sindheducation.gov.pk/SanctionSNE.aspx
Extracted Link: https://checker.sindheducation.gov.pk/ListTraining.aspx
Extracted Link: https://checker.sindheducation.gov.pk/CheckDistance.aspx


In [103]:
import requests
from bs4 import BeautifulSoup

# Target URL
url = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"

# Send GET request
r = requests.get(url)

# Check for successful request
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "lxml")
    
    # Find the specific anchor tag (you can refine this selector based on the page structure)
    specific_link = soup.find("a",class_="btn btn-primary")  # Adjust the text or other attributes as needed
    
    if specific_link:
        # Combine base URL with the relative href
        full_link = requests.compat.urljoin(url, specific_link['href'])
        print("Extracted Link:", full_link)
    else:
        print("Specific link not found.")
else:
    print(f"Failed to fetch the page. Status code: {r.status_code}")


Extracted Link: https://checker.sindheducation.gov.pk/CheckBiometrics.aspx


In [105]:
import requests
from bs4 import BeautifulSoup
import re

# URL to scrape
url = "https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010844&mne=0"

# Send GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract the script tag containing the `gps_location`
    script_tags = soup.find_all('script')
    
    gps_location = None
    for script in script_tags:
        if 'var gps_location' in script.text:
            # Extract the value of the variable using regex
            match = re.search(r"var gps_location\s*=\s*\[\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\];", script.text)
            if match:
                latitude = match.group(1)
                longitude = match.group(2)
                gps_location = (latitude, longitude)
                break
    
    if gps_location:
        print("Extracted GPS Location:", gps_location)
    else:
        print("GPS Location not found.")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

GPS Location not found.


In [107]:
import requests  # Correct module name
from bs4 import BeautifulSoup

url = "https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010844&mne=0"

r = requests.get(url)
print(r)

soup = BeautifulSoup(r.text,"lxml")
print(soup)
np = soup.find("a",class_="btn btn-primary").get("href")
print(np)

#cnp = "https://checker.sindheducation.gov.pk/ListInstitution.aspx"+np

<Response [200]>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
<head><title>
	Schools &amp; Employees Information - SE&amp;LD
</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="Online Checker, Online Biometric Checker, Online Biometric, Online Insitution Checker, Online Institution, School and Employees Information, SE&amp;LD, School Education, School Education &amp; Literacy Department, Government of Sindh, GoS, Literacy Department, Sindh" name="keywords"/><link href="../js/bootstrap.min.css" rel="stylesheet"/>
<script src="http://ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>
<script src="../js/jquery_3.4.1.min.js"></script>
<script src="../js/3.4.0_bootstrap.min.js"></script>
<link href="../assets/global/plugins/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<style type="text/css">
        .table-header-center {
            text-align: center;
        }
    

In [109]:
import requests
from bs4 import BeautifulSoup
import re

# Target URL
url = "https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010844&mne=0"

# Send GET request
response = requests.get(url)
if response.status_code == 200:
    # Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the script containing `initializeMap`
    scripts = soup.find_all('script')
    coordinates = None
    for script in scripts:
        if 'initializeMap' in script.text:
            # Extract the latitude and longitude using regex
            match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
            if match:
                latitude = match.group(1)
                longitude = match.group(2)
                school_name = match.group(3)
                coordinates = (latitude, longitude, school_name)
                break

    if coordinates:
        print("Extracted Coordinates:", coordinates)
    else:
        print("Coordinates not found.")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

Extracted Coordinates: ('24.6524289298985', '68.831448540666', '<strong>Institution Name:GBHSS:GOVERNMENT ENGLISH MEDIUM SCHOOL IN CAMBRIDGE SYSTEM (NURSERY TO O-LEVEL) BADIN (401010844)</strong>')


In [125]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import time
import csv
import re

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Ensure you have the correct WebDriver for your browser
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

# Function to extract table data and institution links
def scrape_table():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    institution_links = []

    for row in rows[1:]:  # Skip the header row
        columns = row.find_elements(By.TAG_NAME, "td")
        if columns:
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)
            # Extract the "Institution ID" link
            link = columns[0].find_element(By.TAG_NAME, "a").get_attribute("href")
            institution_links.append(link)
    
    return data, institution_links

# Function to extract coordinates from the second page
def get_coordinates(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')
        for script in scripts:
            if 'initializeMap' in script.text:
                match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
                if match:
                    latitude = match.group(1)
                    longitude = match.group(2)
                    return latitude, longitude
    return None, None

try:
    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )

    # Initialize lists to store all data and links
    all_data = []
    all_links = []

    # Scrape the first page
    data, links = scrape_table()
    all_data.extend(data)
    all_links.extend(links)

    # Handle pagination
    while True:
        try:
            next_button = driver.find_element(By.LINK_TEXT, "Next")
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
            data, links = scrape_table()
            all_data.extend(data)
            all_links.extend(links)
        except:
            print("No more pages or pagination not found.")
            break

    # Add coordinates to the data
    for i, link in enumerate(all_links):
        latitude, longitude = get_coordinates(link)
        if latitude and longitude:
            all_data[i].extend([latitude, longitude])
        else:
            all_data[i].extend(["N/A", "N/A"])

    # Save data to CSV
    headers = ["Institution ID", "Column 2", "Column 3", "Latitude", "Longitude"]  # Adjust based on your table structure
    with open("school_data_with_coordinates.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(all_data)
    print("Data scraped and saved to school_data_with_coordinates.csv.")
finally:
    driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF72E8CFB05+28789]
	(No symbol) [0x00007FF72E8386E0]
	(No symbol) [0x00007FF72E6D592A]
	(No symbol) [0x00007FF72E72930E]
	(No symbol) [0x00007FF72E7295FC]
	(No symbol) [0x00007FF72E71C6EC]
	(No symbol) [0x00007FF72E74F47F]
	(No symbol) [0x00007FF72E71C5B6]
	(No symbol) [0x00007FF72E74F650]
	(No symbol) [0x00007FF72E76F654]
	(No symbol) [0x00007FF72E74F1E3]
	(No symbol) [0x00007FF72E71A938]
	(No symbol) [0x00007FF72E71BAA1]
	GetHandleVerifier [0x00007FF72EC0933D+3410093]
	GetHandleVerifier [0x00007FF72EC1E7DD+3497293]
	GetHandleVerifier [0x00007FF72EC12A73+3448803]
	GetHandleVerifier [0x00007FF72E997BBB+848171]
	(No symbol) [0x00007FF72E843C3F]
	(No symbo

In [127]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Ensure you have the correct WebDriver for your browser
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

# Function to extract table data and institution links
def scrape_table():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    institution_links = []

    for row in rows[1:]:  # Skip the header row
        columns = row.find_elements(By.TAG_NAME, "td")
        if columns:
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)
            try:
                # Wait for the "Institution ID" link to be clickable and then extract the link
                institution_link = WebDriverWait(row, 10).until(
                    EC.element_to_be_clickable((By.TAG_NAME, "a"))
                ).get_attribute("href")
                institution_links.append(institution_link)
            except:
                institution_links.append("N/A")
    
    return data, institution_links

# Function to extract coordinates from the second page
def get_coordinates(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')
        for script in scripts:
            if 'initializeMap' in script.text:
                match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
                if match:
                    latitude = match.group(1)
                    longitude = match.group(2)
                    return latitude, longitude
    return None, None

try:
    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )

    # Initialize lists to store all data and links
    all_data = []
    all_links = []

    # Scrape the first page
    data, links = scrape_table()
    all_data.extend(data)
    all_links.extend(links)

    # Handle pagination
    while True:
        try:
            # Wait for the "Next" button to be clickable and click it
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Next"))
            )
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
            data, links = scrape_table()
            all_data.extend(data)
            all_links.extend(links)
        except:
            print("No more pages or pagination not found.")
            break

    # Add coordinates to the data
    for i, link in enumerate(all_links):
        latitude, longitude = get_coordinates(link)
        if latitude and longitude:
            all_data[i].extend([latitude, longitude])
        else:
            all_data[i].extend(["N/A", "N/A"])

    # Save data to CSV
    headers = ["Institution ID", "Column 2", "Column 3", "Latitude", "Longitude"]  # Adjust based on your table structure
    with open("school_data_with_coordinates.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(all_data)
    print("Data scraped and saved to school_data_with_coordinates.csv.")
finally:
    driver.quit()

No more pages or pagination not found.


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [131]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re
from bs4 import BeautifulSoup
import requests

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with the appropriate WebDriver for your browser
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

# Function to extract table data
def scrape_table():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    links = []
    for row in rows[1:]:  # Skip header row
        columns = row.find_elements(By.TAG_NAME, "td")
        if columns:
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)
            try:
                # Extract the link for the "Institution ID"
                institution_link = row.find_element(By.TAG_NAME, "a").get_attribute("href")
                links.append(institution_link)
            except:
                links.append("N/A")
    return data, links

# Function to get coordinates from the second page
def get_coordinates(institution_url):
    response = requests.get(institution_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')
        for script in scripts:
            if 'initializeMap' in script.text:
                match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
                if match:
                    latitude = match.group(1)
                    longitude = match.group(2)
                    return latitude, longitude
    return None, None

# Scraping the data and extracting coordinates
all_data = []
try:
    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    # Scrape the first page
    data, links = scrape_table()
    all_data.extend(data)

    # Handle pagination (if available)
    while True:
        try:
            # Locate and click the "Next" button
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Next"))
            )
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
            data, links = scrape_table()
            all_data.extend(data)
        except:
            print("No more pages or pagination not found.")
            break

    # Add coordinates to the data
    for i, link in enumerate(links):
        latitude, longitude = get_coordinates(link)
        if latitude and longitude:
            all_data[i].extend([latitude, longitude])
        else:
            all_data[i].extend(["N/A", "N/A"])

    # Save data to CSV
    headers = ["Institution ID", "Column 2", "Column 3", "Latitude", "Longitude"]  # Adjust based on your table structure
    with open("school_data_with_coordinates.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(all_data)
    print("Data scraped and saved to school_data_with_coordinates.csv.")

finally:
    driver.quit()


MaxRetryError: HTTPConnectionPool(host='localhost', port=9490): Max retries exceeded with url: /session/b744a780a8ac9389da52eea565a5d6f7/element/f.F14642D7B10BFF462D3073531397EA22.d.050E501A7E80F7ED61C6C919F563E0FD.e.59572/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D62E8C9DF0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [135]:
import requests
from bs4 import BeautifulSoup
import re

# Target URL
url = "https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010544&mne=0"

# Send GET request
response = requests.get(url)
if response.status_code == 200:
    # Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the script containing `initializeMap`
    scripts = soup.find_all('script')
    coordinates = None
    for script in scripts:
        if 'initializeMap' in script.text:
            # Extract the latitude and longitude using regex
            match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
            if match:
                latitude = match.group(1)
                longitude = match.group(2)
                school_name = match.group(3)
                coordinates = (latitude, longitude, school_name)
                break

    if coordinates:
        print("Extracted Coordinates:", coordinates)
    else:
        print("Coordinates not found.")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

Coordinates not found.


In [141]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re
import requests
from bs4 import BeautifulSoup

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with the appropriate WebDriver for your browser
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

# Function to extract table data
def scrape_table():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    links = []
    for row in rows[1:]:  # Skip header row
        columns = row.find_elements(By.TAG_NAME, "td")
        if columns:
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)
            try:
                # Extract the link for the "Institution ID"
                institution_link = row.find_element(By.TAG_NAME, "a").get_attribute("href")
                links.append(institution_link)
            except:
                links.append(None)
    return data, links

# Function to get coordinates from the second page
def get_coordinates(institution_url):
    if not institution_url:
        return None, None
    response = requests.get(institution_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')
        for script in scripts:
            if 'initializeMap' in script.text:
                match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
                if match:
                    latitude = match.group(1)
                    longitude = match.group(2)
                    return latitude, longitude
    return None, None

# Scraping the data and extracting coordinates
all_data = []
try:
    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    # Scrape the first page
    data, links = scrape_table()
    all_data.extend(data)

    # Handle pagination (if available)
    while True:
        try:
            # Locate and click the "Next" button
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Next"))
            )
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
            data, links = scrape_table()
            all_data.extend(data)
        except:
            print("No more pages or pagination not found.")
            break

    # Add coordinates to the data
    for i, link in enumerate(links):
        latitude, longitude = get_coordinates(link)
        if latitude and longitude:
            all_data[i].extend([latitude, longitude])
        else:
            all_data[i].extend(["N/A", "N/A"])

    # Save data to CSV
    headers = ["Institution ID", "Column 2", "Column 3", "Latitude", "Longitude"]  # Adjust headers based on table structure
    with open("school_data_with_coordinates.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(all_data)
    print("Data scraped and saved to school_data_with_coordinates.csv.")

finally:
    driver.quit()

No more pages or pagination not found.


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [139]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with the appropriate WebDriver for your browser
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

# Function to extract Institution ID links
def extract_institution_links():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    links = []
    for row in rows[1:]:  # Skip the header row
        try:
            # Locate the "Institution ID" link and extract its href
            link = row.find_element(By.TAG_NAME, "a").get_attribute("href")
            links.append(link)
        except:
            # Skip rows without links
            continue
    return links

try:
    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    # Extract links from the first page
    institution_links = extract_institution_links()

    # Handle pagination (if available)
    while True:
        try:
            # Locate and click the "Next" button
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Next"))
            )
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
            institution_links.extend(extract_institution_links())
        except:
            print("No more pages or pagination not found.")
            break

    # Print the extracted links
    for link in institution_links:
        print(link)

finally:
    driver.quit()

No more pages or pagination not found.
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010544&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010085&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010352&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=301000008&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010141&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010844&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010228&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010428&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=301000010&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010188&mne=0
https://checker.sindheducation.gov.pk/CheckBiometrics.aspx?semisid=401010209&mne=0
https://checker.sindheducation.gov.pk/CheckBiome

In [143]:
with open("C:/path_to_your_folder/school_data_with_coordinates.csv", "w", newline="", encoding="utf-8") as file:


SyntaxError: incomplete input (3667305756.py, line 1)

In [145]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\4587sajid


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re
import requests
from bs4 import BeautifulSoup
import os

# Directory for saving the file
SAVE_DIR = r"C:\Users\4587sajid\Documents"
SAVE_PATH = os.path.join(SAVE_DIR, "school_data_with_coord.csv")

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with the appropriate WebDriver for your browser
driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")

# Function to extract table data
def scrape_table():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    links = []
    for row in rows[1:]:  # Skip header row
        columns = row.find_elements(By.TAG_NAME, "td")
        if columns:
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)
            try:
                # Extract the link for the "Institution ID"
                institution_link = row.find_element(By.TAG_NAME, "a").get_attribute("href")
                links.append(institution_link)
            except:
                links.append(None)
    return data, links

# Function to get coordinates from the second page
def get_coordinates(institution_url):
    if not institution_url:
        return None, None
    response = requests.get(institution_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')
        for script in scripts:
            if 'initializeMap' in script.text:
                match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
                if match:
                    latitude = match.group(1)
                    longitude = match.group(2)
                    return latitude, longitude
    return None, None

# Scraping the data and extracting coordinates
all_data = []
headers = ["Institution ID", "Column 2", "Column 3", "Latitude", "Longitude"]  # Adjust headers based on table structure
rows_processed = 0

try:
    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )
    
    # Open the CSV file in append mode and write headers
    with open(SAVE_PATH, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
    
    # Scrape the first page
    data, links = scrape_table()
    all_data.extend(data)

    # Handle pagination (if available)
    while True:
        try:
            # Locate and click the "Next" button
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Next"))
            )
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
            data, links = scrape_table()
            all_data.extend(data)

            # Process every 100 rows
            if len(all_data) >= rows_processed + 100:
                with open(SAVE_PATH, "a", newline="", encoding="utf-8") as file:
                    writer = csv.writer(file)
                    for i, row in enumerate(all_data[rows_processed:rows_processed + 100]):
                        latitude, longitude = get_coordinates(links[i])
                        if latitude and longitude:
                            row.extend([latitude, longitude])
                        else:
                            row.extend(["N/A", "N/A"])
                        writer.writerow(row)
                rows_processed += 100
                print(f"Saved {rows_processed} rows to CSV.")
        except:
            print("No more pages or pagination not found.")
            break

    # Save remaining rows
    with open(SAVE_PATH, "a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        for i, row in enumerate(all_data[rows_processed:]):
            latitude, longitude = get_coordinates(links[i])
            if latitude and longitude:
                row.extend([latitude, longitude])
            else:
                row.extend(["N/A", "N/A"])
            writer.writerow(row)
    print(f"Data scraped and saved to {SAVE_PATH}.")

finally:
    driver.quit()

No more pages or pagination not found.


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re
import requests
from bs4 import BeautifulSoup
import os

# Directory for saving the file
SAVE_DIR = r"C:\Users\4587sajid\Documents"
SAVE_PATH = os.path.join(SAVE_DIR, "school_data_with_coord.csv")

# Function to set up the WebDriver
def set_up_driver():
    driver = webdriver.Chrome()  # Replace with the appropriate WebDriver for your browser
    driver.get("https://checker.sindheducation.gov.pk/ListInstitution.aspx")
    return driver

# Skip already processed rows
rows_processed = 350  # Set to the number of rows already scraped
all_data = []  # Only keep track of new data

# Open CSV file in append mode
with open(SAVE_PATH, "a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Skip writing headers if continuing


# Function to extract table data
def scrape_table(driver):
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    links = []
    for row in rows[1:]:  # Skip header row
        columns = row.find_elements(By.TAG_NAME, "td")
        if columns:
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)
            try:
                institution_link = row.find_element(By.TAG_NAME, "a").get_attribute("href")
                links.append(institution_link)
            except:
                links.append(None)
    return data, links

# Function to get coordinates with retries
def get_coordinates(institution_url, retries=3):
    if not institution_url:
        return None, None
    for _ in range(retries):
        try:
            response = requests.get(institution_url, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                scripts = soup.find_all('script')
                for script in scripts:
                    if 'initializeMap' in script.text:
                        match = re.search(r"initializeMap\(([-\d.]+),\s*([-\d.]+),\s*['\"](.*?)['\"]", script.text)
                        if match:
                            return match.group(1), match.group(2)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching coordinates: {e}. Retrying...")
            time.sleep(2)  # Wait before retrying
    return None, None

# Main scraping logic
def scrape_data():
    all_data = []
    rows_processed = 0
    headers = ["Institution ID", "Column 2", "Column 3", "Latitude", "Longitude"]

    # Set up WebDriver
    driver = set_up_driver()

    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )
        
        # Open CSV file in write mode and write headers
        with open(SAVE_PATH, "w", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(headers)

        while True:
            # Scrape the current page
            data, links = scrape_table(driver)
            all_data.extend(data)

            # Process and save data in batches of 100 rows
            if len(all_data) >= rows_processed + 100:
                with open(SAVE_PATH, "a", newline="", encoding="utf-8") as file:
                    writer = csv.writer(file)
                    for i, row in enumerate(all_data[rows_processed:rows_processed + 100]):
                        latitude, longitude = get_coordinates(links[i])
                        row.extend([latitude if latitude else "N/A", longitude if longitude else "N/A"])
                        writer.writerow(row)
                rows_processed += 100
                print(f"Saved {rows_processed} rows to {SAVE_PATH}.")

            # Check for next page
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, "Next"))
                )
                next_button.click()
                time.sleep(3)
            except:
                print("No more pages or pagination not found.")
                break

        # Save remaining rows
        with open(SAVE_PATH, "a", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            for i, row in enumerate(all_data[rows_processed:]):
                latitude, longitude = get_coordinates(links[i])
                row.extend([latitude if latitude else "N/A", longitude if longitude else "N/A"])
                writer.writerow(row)
        print(f"Data scraped and saved to {SAVE_PATH}.")

    finally:
        driver.quit()

# Run the scraper
scrape_data()


Error fetching coordinates: HTTPSConnectionPool(host='checker.sindheducation.gov.pk', port=443): Max retries exceeded with url: /CheckBiometrics.aspx?semisid=401010188&mne=0 (Caused by SSLError(SSLWantReadError(2, 'The operation did not complete (read) (_ssl.c:2570)'))). Retrying...
Error fetching coordinates: HTTPSConnectionPool(host='checker.sindheducation.gov.pk', port=443): Read timed out. (read timeout=10). Retrying...
Error fetching coordinates: HTTPSConnectionPool(host='checker.sindheducation.gov.pk', port=443): Read timed out. (read timeout=10). Retrying...
Error fetching coordinates: HTTPSConnectionPool(host='checker.sindheducation.gov.pk', port=443): Read timed out. (read timeout=10). Retrying...
Error fetching coordinates: HTTPSConnectionPool(host='checker.sindheducation.gov.pk', port=443): Read timed out. (read timeout=10). Retrying...
Error fetching coordinates: HTTPSConnectionPool(host='checker.sindheducation.gov.pk', port=443): Read timed out. (read timeout=10). Retrying

In [ ]:
print(f"Saved {rows_processed} rows to {SAVE_PATH}.")